In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/euljiro.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

In [2]:
df_post.head()

,date,location,text
0,2023-11-10T05:20:04.000Z,을지로 3가,.을지로의 상징이 되어버린 올디스😁⠀네온사인이 참 이쁘다
1,2023-11-07T12:29:08.000Z,을지로 3가,일본식 중화요리를 선보이는 을지로 신상 맛집 -각양각색의 맛집들이 모여 있는을지로 ...
2,2023-12-25T09:13:20.000Z,을지로 3가,🎄......#스냅사진 #스냅모델
3,2023-12-23T15:51:57.000Z,을지로 3가,블랜딩밀크가 들어가 고소하고 부드러운 라떼 맛집☕️을지로 인쇄골목 사이에 위치한 #...
4,2023-11-09T02:56:31.000Z,을지로 3가,손에서 컵을 못 놓겠네..☕️...이번에 착용한 데님은 학교 동아리에서 스프레이로 ...


In [3]:
# 상점 데이터 불러오기
gdf_store_euljiro = gpd.read_file('../inputs/stores/store_euljiro.geojson')

# 상호명 없는 상점은 삭제
gdf_store_euljiro = gdf_store_euljiro.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_euljiro = gdf_store_euljiro.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_euljiro = gdf_store_euljiro.rename(columns={'상호명':'store_name'})

In [4]:
gdf_store_euljiro.head()

,store_name,geometry
0,스텔라앤코,POINT (126.98704 37.56715)
1,건축사사무소oneproject,POINT (126.99391 37.56143)
2,삼보국제,POINT (126.98866 37.56180)
3,닷플래너,POINT (126.98840 37.56761)
4,맨즈타겟팅,POINT (126.98694 37.56509)


In [5]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상점 이름 을지인 것은 패스
    if store_name == '을지':
        return 0

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [6]:
gdf_store_euljiro.loc[:,'posting_count'] = gdf_store_euljiro.loc[:,'store_name'].apply(counting_post)

In [7]:
# 포스팅 갯수 기준으로 정렬
gdf_store_euljiro = gdf_store_euljiro.sort_values(by='posting_count', ascending=False)

In [8]:
for i, row in gdf_store_euljiro.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

오늘 283
충무 137
을지로술집 126
아트 94
갤러리 87
블루커피베이크 69
현대 67
선물 59
연락 51
디어 35
충무로맛집 33
광고 30
공간갑 30
36씬 26
미가 25
포인트 25
나무 24
세운 24
디엠 22
만선호프 21
청계 21
스페이스 21
블랙 21
청계 21
에스 21
만선 21
경일옥핏제리아 21
또롱 20
아카 19
일단 19
맛나 18
토마토 18
을지오뎅 17
가제 17
가자 16
가자 16
플러스 16
베로나 15
크사나 15
렉스 14
무한 14
일품 14
힐링 13
힐링 13
82올리언즈 13
낭만 13
힐링 13
리코 13
토리카미 12
레드 12
부장님댁 12
시립서울청소년센터 12
코너숍 11
그린 11
다래 10
안나 10
알렉스룸 10
을지맥옥 10
필스튜디오 10
카다로그 9
줄리아 9
양지 9
혜민당 9
유네사진관 9
양지 9
바나나 9
부자 8
산청숯불가든을지로 8
을지로밑 8
골목집 8
기모 8
애플 8
효계 8
네일 8
언오디너리 7
로바타카미 7
아따 7
오렌지 7
커피사 7
인쇄소 7
안가 7
분카샤 7
베타서비스 7
광장 7
아지트 7
진양 7
타워 7
천국 6
전주집 6
을지1막 6
우르르 6
심플 6
의전방 6
을지깐깐 6
오늘하루 6
시골집 6
타마고 6
우주 6
밸런스 6
재해석 6
을지로라스베가스 6
을지로보석 6
좋은날 6
평균율 6
시골집 6
리치 6
남산 5
일라레플라워 5
시공간 5
워크인 5
토리노라스베가스 5
까이9 5
오덕장 5
을지로골목집 5
곡성 5
원형들 5
돈독 5
주마등 5
남산 5
부스 5
포도 5
아일 5
골드 4
영동 4
시옷시옷음식작업소 4
제로띵스 4
보스 4
단칸 4
선셋레코드 4
한국관 4
남영동양문 4
을지빈 4
을지골뱅이 4
코야 4
어나더레벨 4
포웨이스탑 4
흐드러지다스튜디오 4
문덕 4
원탑 4
푸쉬커피을지 4
잇덴굿 4
콸콸 4
부지 4
사랑방 4
야키토리카미 4
산수갑산 4
보스 4
을지상회 3
명보 3
을지다락 3
피플 3
라이너 

In [9]:
# 상점명 중 너무 보편적이여서 배제할 단어들을 우선 눈으로 리스팅 
list_stopwords = ['오늘', '충무', '을지로술집', '아트', '갤러리', '블루커피베이크', '현대', '선물', '연락', '디어', '충무로맛집', '광고', '포인트', '세운', '나무', '청계', '일단', '맛나', '가자', '무한', '힐링', '낭만', '인쇄소', '남산', '디엠']

In [10]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_euljiro = gdf_store_euljiro.loc[~gdf_store_euljiro.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_euljiro = gdf_store_euljiro.loc[gdf_store_euljiro.loc[:,'posting_count']>0]

In [11]:
# 약 372개 장소가 살아남음
gdf_store_euljiro

,store_name,geometry,posting_count
2419,공간갑,POINT (126.99026 37.56549),30
1409,36씬,POINT (126.99059 37.56582),26
2792,미가,POINT (126.99651 37.56133),25
2818,디엠,POINT (126.99213 37.56401),22
653,만선호프,POINT (126.99177 37.56725),21
...,...,...,...
4507,빠삼,POINT (126.99539 37.56376),1
2332,우진골뱅이,POINT (126.99026 37.56494),1
4441,한울요가,POINT (126.99202 37.56523),1
268,해피클럽,POINT (126.99544 37.56713),1


## pydeck으로 시각화 (탐색용)

In [12]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_euljiro,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [13]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.565, longitude=126.9915, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/euljiro/scatterplot_layer.html")

In [14]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_euljiro,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [15]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/euljiro/heatmap_layer.html")